# <center> Answering Business Questions Using SQL </center>

This project utilizes SQL in order to answer business questions.  We will be examining the Chinook database, which is provided as a SQLite database file called `chinook.db`. It contains information about a fictional digital music shop, the Chinook record store, which is similar to a mini -iTunes store.  The store carries music from around the world and various genres.  We will explore a variety of scenerios in order to maximize profit.   

## Overview of the Data

The Chinook database contains information about the artists, songs, and albums from the music shop, as well as the shop's employees, customers, and the customers' purchases.  All of this is contained in eleven tables.  Let's list all of the tables to view them:    

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [2]:
%%sql

SELECT 
     name,
     type
FROM sqlite_master
WHERE type IN ('table', 'view');

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


# Most Popular Genres

The Chinook record store has just signed a deal with a new record label, so we need to select the first three albums that will be added to the store from a list of four.  We have the artist names and genre of music they produce:
* Regal- Hip-Hop
* Red Tone- Punk
* Meteor and the Girls- Pop
* Slim Jim Bites- Blues

The record label specializes in artists from the USA and have given Chinook some money to advertise the new albums in the USA, so we are interested in finding out which genres will sell the most in the USA.  

Let's commence by determining the number of tracks sold in the USA.

In [3]:
%%sql

WITH customer_quantity AS
    (
    SELECT *
    FROM customer c
    INNER JOIN invoice i ON c.customer_id = i.customer_id
    INNER JOIN invoice_line il ON i.invoice_id = il.invoice_id
    WHERE c.country = 'USA'
    )
    
SELECT SUM(quantity)
    FROM customer_quantity;

 * sqlite:///chinook.db
Done.


SUM(quantity)
1051


Next, we will determine the number of tracks sold in the USA in absolute numbers and in percentages based on genre.

In [4]:
%%sql

WITH 
    customer_quantity AS
    (
    SELECT *
    FROM customer c
    INNER JOIN invoice i ON c.customer_id = i.customer_id
    INNER JOIN invoice_line il ON i.invoice_id = il.invoice_id
    WHERE c.country = 'USA'
    )
    
SELECT g.name genre,
    COUNT(cq.invoice_line_id) number_of_tracks,
    CAST(COUNT(cq.invoice_line_id) AS float) /(
        SELECT COUNT(*) FROM customer_quantity
    ) percentage_sold
    FROM customer_quantity cq 
    INNER JOIN track t ON cq.track_id = t.track_id
    INNER JOIN genre g ON t.genre_id = g.genre_id
    GROUP BY genre
    ORDER BY 2 DESC;

 * sqlite:///chinook.db
Done.


genre,number_of_tracks,percentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Pop,22,0.02093244529019981
Latin,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


Rock is clearly the most popular genre, and encompasses more than half, specifically 53%, of all tracks sold.  Next are Alternative & Punk and Metal, which sell 12.4% and 11.8% respectively.  Blues accounts for 3.4% of total sales and Pop accounts for 2.1%.  Lastly, Hip Hop/Rap accounts for 1.9% of the sales.  Based on this data, we should purchase albums from the following artists for the store:
* Red Tone- Punk
* Slim Jim Bites- Blues
* Meteor and the Girls- Pop

It is important to keep in mind that all of these genres combined only make up 17% of total sales, so we really should be on the lookout for artists and albums from the Rock genre, as they makeup 53% of sales. 

# Employee Sales Performance

Every customer for the Chinook store is assigned to a sales support agent within the company when they first make a purchase.  We will now analyze the purchases of customers belonging to each employee to see if any sales support agent is outperforming or underperforming others.  We will do this by writing a query that finds the total dollar amount of sales assigned to each sales support agent.  We will also list their title and hire date.  

In [5]:
%%sql

WITH customer_total AS
    (
    SELECT SUM(i.total) total,
    i.customer_id,
    c.customer_id,
    c.support_rep_id    
    FROM customer c
    INNER JOIN invoice i ON c.customer_id = i.customer_id
    GROUP BY support_rep_id
    )
    
SELECT e.last_name || ', ' || e.first_name employee_name,
    ROUND(SUM(ct.total), 2) total_sales,
    e.title title,
    e.hire_date hire_date
    FROM employee e 
    INNER JOIN customer_total ct ON ct.support_rep_id = e.employee_id
    GROUP BY 1
    ORDER BY 2 DESC;

 * sqlite:///chinook.db
Done.


employee_name,total_sales,title,hire_date
"Peacock, Jane",1731.51,Sales Support Agent,2017-04-01 00:00:00
"Park, Margaret",1584.0,Sales Support Agent,2017-05-03 00:00:00
"Johnson, Steve",1393.92,Sales Support Agent,2017-10-17 00:00:00


Based on our results, Jane Peacock has the highest amount of sales, \\$1731.51, followed by Margaret Park with \\$1584.00 and Steven Johnson with \\$1393.92.  Jane has generated \\$337.59 more in sales than Steven, which is about 20%.  All of the employees are Sales Support Agents, so there should be no differences in sales due to titles.  What does contribute to the higher sales for Jane compared to the others is that her hire date was in April of 2017 while Margaret's was May and Steven's was October of 2017.  Because Jane has had more experience and time to help customers generate sales, this could largely be the reason why she has the most sales.  

# Analyzing Sales by Country

Our next task is to analyze the sales data for customers from each different country.  We will use the country value from the `customers` table, and ignore the country from the billing address in the `invoice` table.  We will calculate the following data:
* total number of customers
* total value of sales
* average value of sales per customer
* average order value

When a country has only one customer, we will put that country into an "Other" group.  

In [6]:
%%sql

WITH country_other AS
    (
    SELECT
        CASE
            WHEN(
                SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN 'Other'
            ELSE c.country
        END AS country,
        c.customer_id,
        il.*
        FROM customer c
        INNER JOIN invoice i ON c.customer_id = i.customer_id
        INNER JOIN invoice_line il ON i.invoice_id = il.invoice_id
        )
SELECT 
    country,
    total_customers,
    total_sales,
    avg_sales_per_customer,
    avg_order_value
FROM
    (
    SELECT 
        country,
        COUNT(distinct customer_id) total_customers,
        ROUND(SUM(unit_price), 2) total_sales,
        ROUND(SUM(unit_price) / count(distinct customer_id), 2) avg_sales_per_customer,
        ROUND(SUM(unit_price) / count(distinct invoice_id), 2) avg_order_value,
        CASE
            WHEN country = 'Other' THEN 1
            ELSE 0
        END AS sort
        FROM country_other 
        GROUP BY country
        ORDER BY sort ASC, total_sales DESC
        );

 * sqlite:///chinook.db
Done.


country,total_customers,total_sales,avg_sales_per_customer,avg_order_value
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.58,8.72
Other,15,1094.94,73.0,7.45


The data shows that most of the sales and customers originate from USA, which accounts for 13 customers, \\$1040.49 in total sales, \\$80.04 in average sales per customer, and a \\$7.94 average order value.  The next country, Canada, has 8 total customers and about half the total number of sales, \\$535.59.  Next is Brazil wth 6 total customers and \\$427.68 in sales.  India is the last country shown with 2 customers and \\$183.15 in sales.  There are 15 countries where there is only one customer with a total amount of sales at \\$1094.94.  It is clear that the USA is the most profitable out of all of the countries, yet all 15 of the "Other" countries generate slightly higher sales than the USA, so they are important.  

Also, there may be opportunity in the following countries:
* Czech Republic
* United Kingdom
* India

The average sales per customer for these countries is the highest, especially the Czech Republic.  It is worth keeping in mind that because the amount of data from each of these countries is relatively low, we have to be cautious about spending too much money on new marketing campaigns.  We would need a larger sample size to give us higher confidence.  A better approach might be to run small campaigns in these countries to collect and analyze the new customers.  This way we can see if these trends hold true with the new customers.  



# Album vs Individual Tracks

The Chinook store is set up in a way that allows customers to puchase in one of two ways:
* purchase an entire album
* puchase a collection of one or more individual tracks

The store does not allow customers to puchase an album and then add individual tracks to the same purchase, but they can choose the album by purchasing all tracks separately.  When customers purchase albums, they are charged the same price as if they had purchased each of those tracks separately.  

Management is currently considering changing their purchasing strategy to save money.  The strategy they are considering is to puchase only the most popular tracks from each album from record companies, instead of purchasing eery track from an album.  

We will find out hwat percentage of puchases are indvidual vs entire albums, so management can use this datat to understand the effect this decision might have on overall revenue.  

In [7]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY invoice_id
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent_of_invoices
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

 * sqlite:///chinook.db
Done.


album_purchase,number_of_invoices,percent_of_invoices
no,500,0.8143322475570033
yes,114,0.18566775244299674


Album purchases account for 18.6% of the purchases, while single track purchases account for 81.4% of purchases.  Based on this data, it would not be wise to purchase only select tracks from albums from record companies as there is potential to lose 1/5 of revenue.  

## Most Popular Artists

Next, we will explore which artist is used in the most playlists.  This type of knowledge would benefit the advertising and promotions departments, allowing them to attract more customers.  

In [8]:
%%sql

WITH artist_playlist AS
    (
    SELECT ar.*,
       al.artist_id,
       t.album_id,
       t.track_id,
       p.*
    FROM artist ar 
    INNER JOIN album al ON ar.artist_id = al.artist_id
    INNER JOIN track t ON al.album_id = t.track_id
    INNER JOIN playlist_track p ON t.track_id = p.track_id
    )
    
SELECT ap.name artist_name,
        SUM(ap.playlist_id) number_of_playlists
    FROM artist_playlist ap
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 10;

 * sqlite:///chinook.db
Done.


artist_name,number_of_playlists
Iron Maiden,209
Led Zeppelin,136
Deep Purple,134
Metallica,112
U2,106
Ozzy Osbourne,84
Pearl Jam,70
Accept,57
AC/DC,57
Various Artists,56


Iron Maiden is clearly the top artist, who is featured in 209 playlists.  Next would be Led Zeppelin and Deep Purple with 136 and 134 playlists repectively.  It would be beneficial to advertise these artists more in order to generate more sales.  